# Classification Problem
Next I will develop a Logistic Regression model to predict different classes. More specifically Logistic Regression is used to estimate the probability that an instance, element or observation belongs to a certain class. The use of one of the most popular collections of information for the purpose of classification is the Titanic dataset and the model I have developed will be submitted to Kaggle's 'Titanic - Machine Learning from Disaster' competition.

The purpose of this model is to identify if these passengers 'Survived' or 'Not' which will involve creating a target output column populated with simple binary results of '1' or '0'.

1. Explore and clean the data
2. Split data into train / validation / test
3. Fit an initial model and evaluate
4. Tune hyper parameters
5. Evaluate on validation set
6. Final model selection and evaluation on test set

## Import the Python Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_curve, precision_recall_curve
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

## Ingest the Data

In [ ]:
titanic = pd.read_csv('C:/Users/lynst/Documents/GitHub/machine-learning-projects/supervised-learning/regression/logistic-regression/titanic_data.csv')
titanic.head()

In [ ]:
# check the column names and data types
titanic.info()

So I can determine there are a total of 183 entries in this dataset. Initial thoughts are that it might be worth using a more comprehensive dataset, one which might contain the full list of passengers (1309) rather than just a subset (183). This is the most comprehensive list available for the purpose of this exercise that I can find, although estimates for the total number of passengers and crew members are thought to be in the region of 2220. The most comprehensive datasets might be Encyclopedia Titanica and Wikipedia, both of which can be found online.

In [ ]:
# importing once again
titanic = pd.read_csv('C:/Users/lynst/Documents/GitHub/machine-learning-projects/supervised-learning/regression/logistic-regression/titanic.csv',
                     header=0,
                     names = ['PassengerId','Survived','Pclass','Name','Sex','Age','SibSp','Parch','Ticket','Fare','Cabin',
                              'Embarked','WikiId','Name_wiki','Age_wiki','Hometown','Boarded','Destination','Lifeboat','Body',
                              'Class'])
titanic.head()

The keys can be discovered as follows:

In [ ]:
titanic.keys()

This provides a list, or 1-d array of the attributes in my dataset.

## Feature Selection
Removing unwanted columns and rows and feature engineering is the next important step. Straight away I can see the second dataset I have imported from Kaggle which I have named 'titanic.csv' has a more comprehensive number of entries but also contains 21 columns as opposed to just 12 in the first set. Time to establish which of these columns will be kept or removed using some dimensionality reduction and combination, before establishing what is to be included in a Pandas DataFrame table and target Series.

In [ ]:
# check the column names and data types
titanic.info()

I can remove 'PassengerId', 'Name', 'Age', 'Ticket', 'Embarked', 'WikiId', 'Name_wiki', 'Hometown', 'Destination', 'Lifeboat', 'Body', 'Fare' and 'Class' which will significantly reduce clutter in my table as these features provide no causal relationship with passenger Survival, some of which also represent duplicated information such as passenger class 'Pclass' and 'Class'. This initial step of reducing the size helps provide a much more useful dataset overall.
Next, let's determine the index and column values.

In [ ]:
titanic.shape

This tells me there are 1309 rows and 21 columns. Also, another way to find the number of entries or range of the index would be:

In [ ]:
titanic.index

So the index starts at 0 and ends at 1308, a total of 1309 passengers (not including crew members), but in terms of the data entries in this table only 891 are labeled with target predictions. The model will be applied to the labeled data first, followed by the unlabeled data (418 entries).

### Some Descriptive Stats
Several of these values can come in handy in case I need to impute any averages for missing values later. Bear in mind this only contains data for columns with numeric values!

In [ ]:
titanic.describe()

In [ ]:
titanic.columns

If I want to visualize these relationships between independent variables themselves and between the dependent variable and independent variables as individual scatter plots, this can be achieved using Seaborn's pairplot() method and specifying the dataset as the argument.

In [ ]:
sns.pairplot(titanic)

### Dropping Columns
Store a copy of these columns in a new variable or dataset so I don't overwrite the information. The visual scatterplot matrix above tells me that certain attributes are highly positively correlated and can be dropped immediately from the dataset..

In [ ]:
titanic_new = titanic[['PassengerId','Survived','Pclass','Name','Sex','Age','SibSp','Parch','Ticket','Fare','Cabin','Embarked',
                       'WikiId','Name_wiki','Age_wiki','Hometown','Boarded','Destination','Lifeboat','Body','Class']]
titanic_new.head()

The easiest way to drop the columns not required is to create a new subset of data (2d array) with the specified columns to be used.

In [ ]:
titanic_reduced = titanic_new[['Survived','Pclass','Sex','SibSp','Parch','Cabin','Age_wiki','Boarded','Fare']]
titanic_reduced.head()

I am using 'Age_wiki' from the Wikipedia web site which appears to be a much more comprehensive set of data than that of the 'Age' column. The 'Boarded' column has nominal data which I will endeavour to convert to numeric values so each port a passenger embarks from will be represented by a number instead. This will help provide more uniform data types. 'Sex' can also be converted to 0's or 1's for the purpose of this exercise and the 'SibSp' and 'Parch' columns can be combined using feature extraction to concatenate the size of families into a new series. The 'Cabin' data will be converted to binary integer values for simplicity and the 'Pclass' (Passenger Class) observations are already denoted as integer values. These appear to have a significant impact on passenger survival so I'm including them.

Reducing the number of features is called dimensionality reduction and is an important technique used to achieve comparable results in a much faster time frame (with little benefit to performance accuracy), but generally works better with much larger datasets. The removal of all the unwanted columns contained in the modified dataset variable helps speed the model up.

In [ ]:
# Partial or subset of titanic dataframe
df = pd.DataFrame({
    'Survived': pd.Series(titanic_reduced['Survived']),
    'Pclass': pd.Series(titanic_reduced['Pclass']),
    'Sex': pd.Series(titanic_reduced['Sex']),
    'SibSp': pd.Series(titanic_reduced['SibSp']),
    'Parch': pd.Series(titanic_reduced['Parch']),
    'Cabin': pd.Series(titanic_reduced['Cabin']),
    'Age_wiki': pd.Series(titanic_reduced['Age_wiki']),
    'Boarded': pd.Series(titanic_reduced['Boarded']),
    'Fare': pd.Series(titanic_reduced['Fare'])
})

df

Which columns or features are left now?

In [ ]:
df.columns

## Clean the Data
### Missing Values
Next it's really important to remove or impute any Null or missing values. This depends on any row values which are missing and also on the data type for each column. Calculating the total number of missing or Null values across the entire 'titanic' dataset gives:

In [ ]:
titanic_missing = pd.isnull(titanic).sum()
print(titanic_missing)

More specifically, to narrow my workable dataset down and find the total number of missing values from the entire dataset:

In [ ]:
df_missing = pd.isnull(df).sum()
print(df_missing)

Next it becomes useful to determine if these missing values occurred because they weren't recorded or because there was no information for them? Assessing this output I can determine that the null values in 'Cabin' simply represent those who did not have a cabin for sleeping quarters and so these would not have been recorded. These passengers would have traveled in other areas of the ship so it's important not to drop these values as they represent important data and account for over three quarters of the overall number of passengers in this particular set. 

There are also 5 null values for the 'Boarded' column so for whatever reason these passengers did not have their boarding locations recorded. It's impossible to really know which port location these individuals departed from so I can either leave the values as NaN or remove each of these 5 entries as a value should exist if they boarded legally and other attributes were recorded, e.g. Name, Class, Cabin or even Age.

### Null Values for Age
The 'Age-wiki' feature records the ages provided by passengers when purchasing their tickets so it was likely based on the D.O.B. in their travel documents or passports. Taking a look at the total number of Null or missing values for the 'Age_wiki' column first:

In [ ]:
num_age_null = df['Age_wiki'].isnull().sum()
print(num_age_null)

Identifying each row in the dataframe which contains a null value for 'Age_wiki'.

In [ ]:
df[df['Age_wiki'].isnull()]

I can make a decision whether to include these 7 passengers and merely impute some average age for their respective 'Sex', impute an average based on the overall mean for both genders, or remove them completely. Seeing as the majority of information for each of these passengers (roughly 4/7ths to 5/7ths) is present I would prefer to keep these entries, so imputing mean values for age based on the individuals sex may be a reasonably accurate average.

The mean age for everyone, regardless of sex is 29.88 according to the describe() method above. Another way to find the overall average age is:

In [ ]:
df.groupby(df['Age_wiki'].isnull()).mean()

This overall mean or average may not be as accurate as calculating the average age for both male and female passengers and imputing them into the 7 missing values.

### Calculate Average Age
Calculating the average age for male and female passengers in the table can be done by summing each individual age (by sex) and dividing by the total number of male or female passengers respectively.

In [ ]:
# find the total count of each of the two unique values in the Sex column (total male or female passengers)
df['Sex'].value_counts().unique()

This is based on all 1309 passengers.

In [ ]:
import seaborn as sns

df.Survived[df.Sex == 'male'].value_counts().plot(kind='bar', alpha=0.5, color='teal')
plt.title("Male Survival")
# create style
sns.set_style("ticks")

In [ ]:
df.Survived[df.Sex == 'female'].value_counts().plot(kind='bar', alpha=0.5, color='pink')
plt.title("Female Survival")
# create style
sns.set_style("ticks")

So having determined the unique classes within the 'Sex' column I can further identify the number of Males and Females who survived or not. By taking the 'Survived' column and sub-dividing it according to gender it displays how women were far more likely to have survived the Titanic disaster based on the predictor variables included with this dataset.

Next I want to group each category of male and female and store them in a variable called 'gender'.

In [ ]:
gender = df.groupby(df['Sex'])
gender.head()

Checking the first few entries for both sexes:

In [ ]:
# Import Library
import seaborn as sns
import matplotlib.pyplot as plt

# Countplot
sns.catplot(x ="Sex", hue ="Survived",
kind ="count", data = df)

Now there are two variables, one with all the male and one with all the female passengers in the Titanic dataset. There are a total of 843 male and 466 female passengers.

The next step is to add these totals together.

In [ ]:
male_total = 843
female_total = 466
total_passengers = male_total + female_total
total_passengers

Summing the total of all ages for all the passengers:

In [ ]:
sum_age = df['Age_wiki'].sum()
print(sum_age)

And dividing by the total number of passengers:

In [ ]:
age_ave = sum_age / total_passengers
age_ave

So the overall average age for all passengers calculates to just over 29 years old. Using the describe method to check this gives:

In [ ]:
titanic.describe()

So the first item to notice is that only numeric data appears to have been captured which will need to be fixed soon, but the answer I was looking for now, the mean age found under the 'Age-wiki' column is 29.415829 which is close to the value just calculated of 29.258525, but not identical.

Next, to see the average ages for both male and female classes:

In [ ]:
df.groupby(by='Sex')['Age_wiki'].mean()

So this produces the mean Age by Sex. 

What if I wanted to find an average age just for the missing values? There are a total of 7 missing age values in the entire dataset, 4 male and 3 female. Imputing a value of 29 for the missing 'male' age and 28 for 'female' would be the most accurate solution but instead, the mean age value for all passengers will be imputed.

In [ ]:
df.isnull().sum()

### Imputing Missing Age Values
Using the 'impute' library rather than using fillna().

In [ ]:
from sklearn.impute import SimpleImputer

# Use SimpleImputer function to fill in missing values
imputer = SimpleImputer(strategy='mean', missing_values=np.nan)
age_imputer = imputer.fit(df[['Age_wiki']])
# now the transform method
df['Age_wiki'] = age_imputer.transform(df[['Age_wiki']])
df['Age_wiki'].head()

All 'Age_wiki' observations are entered as floats but the data type needs to be changed to integer (representing years) which is technically more correct as a unit of measurement.

In [ ]:
df['Age_wiki'] = df['Age_wiki'].astype('int')
df['Age_wiki'].head()

So checking the first missing age value to see what value it contains now:

In [ ]:
df.Age_wiki[42:43]

The average age of 29 has been imputed. Another of the missing values was located at index 1041:

In [ ]:
df.Age_wiki[1041:1042]

This also appears to be correct. 

To check if there are any more null values now they have been replaced with the mean age.

In [ ]:
# check if Age_wiki has any more null values
df['Age_wiki'].isnull().sum()

This tells me all the null values in the Age column have been replaced properly. 

Now the missing values have been dealt with, what about the string and categorical variables in the dataset? An important consideration to make when using visualizations would be the data types involved. For example, information can be split into numeric (quantitative) data and categorical (qualitative) data. Categorical data values could be classified as Binary (such as the target outcome 'Survived', or 'Sex'), Nominal (such as 'Cabin', or 'Boarded'), perhaps even Ordinal (such as 'Pclass'). 'Age_wiki' contains continuous values and the rest such as 'SibSp' (number of Siblings or Spouse) and 'Parch' (number of Children accompanied by Parents) are discrete integer values.

Having established the different different data types the next step is to convert the string objects into numeric types, starting with the data in 'Cabin' first.

### Null Values for Cabin
Checking the total number of Null values for the 'Cabin' feature:

In [ ]:
num_cabin_null = df['Cabin'].isnull().sum()
print(num_cabin_null)

### Convert the Cabin Feature
Next, I want to address the issue relating to Cabin data. Because each passenger was assigned a Cabin number which is just an alpha-numeric string type, I would prefer to convert all entries for Cabin to a straight forward integer. Assigning a value of 1 for the presence of a cabin number and 0 for someone without.
#### Creating a Binary Indicator for Cabin

In [ ]:
# Overwrite the Cabin values having converted them and store them in the same name
df['Cabin'] = np.where(df['Cabin'].isnull(), 0, 1)
df['Cabin'].head()

Now the 'Cabin' data has been converted into binary numeric values equivalent to 'cabin' or 'no cabin' and even though there are 1014 passengers who were not assigned cabins, this data will remain.

In [ ]:
df.head()

### Convert the Sex Feature
The male entries will be assigned a value of 1 and female, 0. Viewing the total number of Male passengers who didn't survive (0.0), or did survive (1.0), these can be switched to integer values also. This can be repeated for the Female passengers also.

In [ ]:
gender_dict = {'male': 1, 'female': 0}
df['Sex'] = df['Sex'].map(gender_dict)

Check the dataframe again!

In [ ]:
df.head()

### Null Values for Fare
There is only one missing value. I could replace the missing fare with the overall mean or median value, but I have opted to adjust the strategy argument in the SimpleImputer() method with the 'most_frequent' value.

In [ ]:
from sklearn.impute import SimpleImputer

# Use SimpleImputer model to fill in missing entries with most frequent value
imputer = SimpleImputer(strategy='most_frequent', missing_values=np.nan)
fare_imputer = imputer.fit(df[['Fare']])
df['Fare'] = fare_imputer.transform(df[['Fare']])

#imputer = fare_imputer.fit_transform(fare_imputer.values.reshape(-1,1))
df[['Fare']].head()

In [ ]:
df.Fare.fillna(df.Fare.mean(), inplace=True)

### Null Values for Boarded
The last categorical feature which needs changing is the 'Boarded' column. Checking if there are null values in 'Boarded':

In [ ]:
df['Boarded'].isnull().sum()

Which 5 indexed rows are missing in the 'Boarded' column?

In [ ]:
# list index values in relation to this column which are missing
boarded_values_null = df[df['Boarded'].isnull()]
boarded_values_null

Because there are only five missing entries, it makes more sense to drop these rows from the database completely as it would be difficult to calculate or impute values for the departure port.

This means when it comes to preparing the dataframe to be used for our predictors, the row index numbers for these entries should be removed completely, thus deleting the information across all the columns.

In [ ]:
df.drop([347, 557, 1041, 1048, 1228], axis=0, inplace=True)
df

So the dataframe has now been reduced from 1309 to 1304 entries.

In [ ]:
# check if Boarded has any more missing values
df['Boarded'].isnull().sum()

An alternative approach would be to replace the missing values with the most frequently occurring entries using the fillna() method. Having observed these five entries in the source dataset there doesn't appear to be much information in the other columns so they probably wouldn't add much value to our model.

To identify which columns are categorical features:

In [ ]:
# Get list of categorical variables from df
cat_vars = (df.dtypes == 'object')
object_cols = list(cat_vars[cat_vars].index)

print("Categorical variables:")
print(object_cols)

### Convert the Boarded Feature
Because 'Boarded' values are just 'nominal', it doesn't make sense to use 'ordinal' encoding to categorize each departure location as there is no precedent regarding importance. A more appropriate way to convert the categorical entries for the Boarded column is to change its entries into numeric values using 'One-Hot Encoding'.

#### Creating a One-Hot Encoding Matrix for Boarded
There are four possible states for the 'Boarded' column including Belfast, Queenstown, Southampton and Cherbourg which were the only departure locations listed. Each of these entries can be assigned a value of 1,2 or 3. This can be achieved using a technique called one-hot encoding.

Check the values in the specific series I want.

In [ ]:
boarded_cat = df[['Boarded']]
boarded_cat.head()

Apply the 'One-Hot Encoder' class from the sci-kit learn library.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

one_hot_encoder = OneHotEncoder(categories='auto', drop=None, sparse=True, dtype='int64', handle_unknown='error')
boarded_cat_1hot = one_hot_encoder.fit_transform(boarded_cat)
boarded_cat_1hot.toarray()

Check to see if the Boarded column has changed to numeric values.

In [ ]:
df.head()

So the Boarded column doesn't appear to have changed. 

#### Use the Column Transformer
This will extrapolate the existing data and extend the data into new columns.

In [ ]:
from sklearn.compose import make_column_transformer

transformer = make_column_transformer(
    (OneHotEncoder(), ['Boarded', 'Sex']),
    remainder='passthrough')

transformed = transformer.fit_transform(df)
updated_df = pd.DataFrame(transformed, columns=transformer.get_feature_names_out())
updated_df.head()

Tidying up the column names in the new dataframe. Make sure you initiate the 'self' class argument by typing in the name of the dataframe itself (updated_df), or else the rename() method won't work.

In [ ]:
updated_df = pd.DataFrame.rename(updated_df, columns={'onehotencoder__Boarded_Belfast':'Boarded_Belfast',
                            'onehotencoder__Boarded_Cherbourg':'Boarded_Cherbourg',
                            'onehotencoder__Boarded_Queenstown':'Boarded_Queenstown',
                            'onehotencoder__Boarded_Southampton':'Boarded_Southampton',
                            'onehotencoder__Sex_0':'Female',
                            'onehotencoder__Sex_1':'Male',
                            'remainder__Survived':'Survived',
                            'remainder__Pclass':'Pclass',
                            'remainder__SibSp':'SibSp',
                            'remainder__Parch':'Parch',
                            'remainder__Cabin':'Cabin',
                            'remainder__Age_wiki':'Age_wiki',
                            'remainder__Fare':'Fare'})
updated_df.head()

Now all the data has been converted to numeric values into a more comprehensive set the performance of the model will be improved dramatically. One interesting visual summarizing the data dispersion of all the features would be a hist plot. This provides a frequency distribution of discrete and continuous variables. 

For example, we can qualify discrete variables as those which occupy a specific number of states determined by 1 / k where k equals the number of possible outcomes within this universal set. Determining the likelihood of each state occurring is calculated using a probability mass function. This would include 'Survived', 'Pclass', 'Sex', 'SibSp', 'Parch' and 'Cabin' below.

Continuous variables have an infinite number of possible values and act as real numbers. These values are calculated using a probability density function which does not provide the probability of a specific state, for example, 'Age_wiki' and 'Fare'.

In [ ]:
df.hist(bins=20, figsize=(20,10))          # Boarded is not included because it contains string data types

Taking a quick glance at the frequency distributions for each column reveals some interesting information. For example there are 2 distinct categories for survival, there don't appear to be any Sibling-Spouse groups larger than 8 in total, there's an average age of roughly 20-21 years, there are 3 distinct passenger classes (1st, 2nd and 3rd), an overwhelming number of People travelling without minors and much smaller numbers of Parents travelling with just 1 or 2 children. Finally, the vast majority of fares appear to be below the 25.0 mark, but there were some almost approaching 275.0. I'm not sure if these are Schillings, Guineas or Pounds Sterling, but this data can be further researched.

When it comes to visualizing these different types of data it is generally better to use scatter and line plots for numeric data, but for categorical data, frequency distributions, bar charts and histograms may be a better approach for viewing different classes or sub-sets of values.

The following Seaborn plots provide a breakdown of the relationship between the three categorical variables and the rate of survival in each. One thing to note is that the survival rate for passengers with a cabin approximates 0.4 (40%), so it would seem that 0.6 (60%) of those without a cabin had survived.

In [ ]:
for i, col in enumerate(['Cabin', 'Sex', 'Boarded']):
    plt.figure(i)
    sns.catplot(x=col, y='Survived', data=df, kind='point', aspect=2, color="lightseagreen")

So I have the passenger survival rates from the different locations above and the percentage number of those passengers who embarked from each port, below.

In [ ]:
df.Boarded.value_counts(normalize=True).plot(kind="bar", alpha=0.6, color="darkorange")
plt.title("Boarded Location")
plt.show()

Total passengers by Boarding location:

In [ ]:
df[0:891]['Boarded'].value_counts()

Group the different Boarding locations by the column I want information returned on which is 'Survived'.

In [ ]:
df[0:891].groupby(['Boarded'], as_index=False)['Survived'].value_counts()

To visualize this.

In [ ]:
# Stacked barplot with pandas
Survived = df.loc[df['Survived']==1, :]['Boarded'].value_counts()
Died = df.loc[df['Survived']==0, :]['Boarded'].value_counts()
df_plot = pd.DataFrame([Survived,Died])
df_plot.index = ['Survived','Died']

# Plot
df_plot.plot(kind='bar',stacked=True, title='Survival Rate by Boarding Location')

This produces an interesting plot because straight away I am able to see that more people died from Southampton than the entire number of those who survived and nobody from Belfast survived at all.

If I flip the x-axis variables to contain the 'Boarded' locations I can compare the number of passengers who embarked from these locations to their actual survival rate in a stacked bar chart.

In [ ]:
# Stacked barplot with pandas
Southampton = df.loc[df['Boarded']=='Southampton', :]['Survived'].value_counts()
Cherbourg = df.loc[df['Boarded']=='Cherbourg', :]['Survived'].value_counts()
Queenstown = df.loc[df['Boarded']=='Queenstown', :]['Survived'].value_counts()
Belfast = df.loc[df['Boarded']=='Belfast', :]['Survived'].value_counts()

df_plot = pd.DataFrame([Southampton,Cherbourg,Queenstown,Belfast])
df_plot.index=['Southampton','Cherbourg','Queenstown','Belfast']
df_plot.plot(kind='bar',stacked=True, color=['tomato','lightseagreen'], title='Survival by Boarding Port Location')

0.0 (tomato color) represents passengers who died and 1.0 (lightseagreen) those who survived in terms of the legend. It looks as if at least two-thirds to three-quarters of all passengers did not make it. At least 55% of overall passengers who survived had embarked from Cherbourg, at least 65% of passengers who were allocated a cabin survived and an overwhelming number of survivor's were Female.

Another important table I like is the correlation matrix which does a great job of describing the relative relationships between all the different variables.

In [ ]:
correlation = updated_df.corr()
correlation

Presenting a correlation matrix in visualized form we can see the darker squares represent higher positive correlation.

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(correlation, annot=True, cmap='BuPu')

Looking at the Survived column it's easy to see that the highest positive correlation is 'Female', so merely being a Women would have the greatest survival rate, more than any other factor. The highest negatively correlated effect belonged to the 'Male' category. This had a much greater impact on survival rates than passenger class (Pclass) or even the fare paid so those who paid more for a first class ticket weren't necessarily secured a place on a lifeboat.

### Grouping Data Together
Taking a look at the average values for each feature based on their survival.

In [ ]:
updated_df.groupby('Survived').mean()

### Combine SibSp and Parch
Creating a for loop to iterate through both columns and plotting them categorically.

In [ ]:
for i, col in enumerate(['SibSp', 'Parch']):
    plt.figure(i)
    sns.catplot(x=col, y='Survived', data=updated_df, kind='point', aspect=2, )

An extremely useful procedure when it comes to feature engineering involves combining columns, in this case 'SibSp' and 'Parch' into a new feature called 'Family_count'. It serves a similar purpose which is to provide counts or frequency of individuals traveling with their Parents or Brothers and Sisters, combining the values together and producing dimensionality reduction by reducing the number of columns and overal noise in the model.

In [ ]:
updated_df['Family_Count'] = updated_df['SibSp'] + updated_df['Parch']
sns.catplot(x='Family_Count', y='Survived', data=updated_df, kind='point', aspect=2, color="coral")

Immediately from this feature-engineered column called 'Family_Count' it's possible to see that an individual had a far greater chance of survival given a lower family count.

In [ ]:
updated_df['Family_Count'] = updated_df['SibSp'] + updated_df['Parch']
updated_df.drop(['SibSp', 'Parch'], axis=1, inplace=True)
updated_df.head()

A plot of Age and Survival Rate.

In [ ]:
updated_df.Age_wiki.value_counts(normalize=True).plot(kind="kde", color="turquoise")
plt.title("Age and the Probability of Survival")
plt.show()

This shows a meaningful relationship across the Age spectrum. The older ages were less likely to survive and as the age diminishes, the likelihood goes up to between 20% and 60% for those less than the age of about 13.

The next step is to prepare the dataset with the correct number of total labeled entries. I can either change the data at source and slice it using Excel, or alternatively slice the data in Python to only include the first 891 passengers in the training data because these are the only results which are labeled. The reason this needs to be done is because of the risk of feeding inaccurate and unlabeled data back into the model (i.e. entries from 892 to 1304). Using un-labeled data will introduce bias into the classification results.

## Plot Continuous Features
These are separate from discrete values and deserve some attention.

In [ ]:
for i in ['Fare','Age_wiki']:
    died = list(updated_df[updated_df['Survived'] == 0][i].dropna())
    survived = list(updated_df[updated_df['Survived'] == 1][i].dropna())
    xmin = min(min(died), min(survived))
    xmax = max(max(died), max(survived))
    width = (xmax - xmin) / 40
    sns.distplot(died, color='lightseagreen', kde=False, bins=np.arange(xmin, xmax, width))
    sns.distplot(survived, color='orange', kde=False, bins=np.arange(xmin, xmax, width))
    plt.legend(['Did not survive', 'Survived'])
    plt.title('Histogram Showing Survival Rate v {}'.format(i))
    plt.show()

This tells me that a far greater number of passengers in the 20-30 year age category did not survive relative to other age groups. Also, it appears that those who paid a lot more had better chances of survival.

What's the relationship between the continuous variables? A scatter plot can be used to describe this.

In [ ]:
price = sns.scatterplot(data=updated_df, x=updated_df['Age_wiki'], y=updated_df['Fare'], color="orange")

This explains the range of fares paid by passengers according to age.

## Row Selection
This is where I determine the number of instances to train for model selection and at this stage it's best to employ the use of a Train, Test, Split algorithm using the labeled data (rows 1 to 889) which contains values for the target feature entitled 'Survived'. The model will fit to and learn from this data, then I can evaluate the efficiency of the chosen model against the unseen (un-labeled) data, rows 892 to 1304 which don't contain target values for the 'Survived' column. It's important to note that because I have already pre-processed the entire dataset from titanic.csv, I will have to split the data into subsets for 'labeled' and 'unlabeled' portions and apply the LogisticRegression() model to each separately.
### Nature of the Data
Change all features to integer values except for the 'Fare' column.

In [ ]:
# change values to integers except for Fare and Survived columns
updated_df['Boarded_Belfast'] = updated_df['Boarded_Belfast'].astype('int')
updated_df['Boarded_Cherbourg'] = updated_df['Boarded_Cherbourg'].astype('int')
updated_df['Boarded_Queenstown'] = updated_df['Boarded_Queenstown'].astype('int')
updated_df['Boarded_Southampton'] = updated_df['Boarded_Southampton'].astype('int')
updated_df['Female'] = updated_df['Female'].astype('int')
updated_df['Male'] = updated_df['Male'].astype('int')
updated_df['Pclass'] = updated_df['Pclass'].astype('int')
updated_df['Cabin'] = updated_df['Cabin'].astype('int')
updated_df['Age_wiki'] = updated_df['Age_wiki'].astype('int')
updated_df['Family_Count'] = updated_df['Family_Count'].astype('int')

updated_df.head()

In [ ]:
updated_df.columns

### Labeled Dataset Only
This set is reduced in size. It only contains instances where the target response was labeled which means only supervised learning algorithms can be applied before evaluating the result. An important point to remember here is that two 'Boarded' entries were removed between the index values of 0 and 891 (giving a total of 889 instances), so the new range of labeled entries will be between 0 and 889. The unlabeled entries span from index locations 889 to 1304.

The unlabeled portion of data residing between index locations 889 and 1304 would require a different type of machine learning model, or un-supervised algorithms such as visualization through clustering techniques, dimensionality reduction or associative rule modeling. See my k_means_clustering.ipynb model... 

In [ ]:
# removing the 'Survived' column from the predictors DataFrame variable X
X = updated_df[:889].drop('Survived', axis=1).copy()
# assigning this dropped column to the target Series variable y
y = updated_df[:889]['Survived']

To clean up the dependent variable information, convert the y values to integers.

In [ ]:
y = pd.Series(y).astype(int)

Checking the first few instances.

In [ ]:
X.head()

In [ ]:
y.head()

This is correct! The first Null value should appear at index location 889, so there are a total of 888 labeled entries. Converting these dataframes to separate csv files so they can be viewed in Microsoft Excel if required.

In [ ]:
X_to_csv = X.to_csv(r'C:/Users/lynst/Documents/GitHub/machine-learning-projects/supervised-learning/regression/logistic-regression/X.csv', index=False, header=True)
X

And the target labels.

In [ ]:
y_to_csv = y.to_csv(r'C:/Users/lynst/Documents/GitHub/machine-learning-projects/supervised-learning/regression/logistic-regression/y.csv', index=False, header=True)
y

Looking at the new shape of the predictor dataframe:

In [ ]:
X.shape

This is better! Now I have the labeled instances for my independent variables, it's time to train and fit the data which means it needs to be split into training and validation sets before evaluating the model's accuracy.

### Split the Data

Having looked at the source data file, it hasn't been split into training or test data yet.

#### Single Holdout Set
Start with a single holdout validation, or test set to evaluate how well the model generalizes.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

So 889 labels (y) are split into training (533 entries), validation (178 entries) and test sets (178 entries). 

In [ ]:
print(len(y), len(y_train), len(y_val), len(y_test))

Next I would like to convert my cleaned, split data into separate train, validation and test set CSV files for preservation.

In [ ]:
X_train.to_csv('C:/Users/lynst/Documents/GitHub/machine-learning-projects/supervised-learning/regression/logistic-regression/train_features.csv', index=False)
X_val.to_csv('C:/Users/lynst/Documents/GitHub/machine-learning-projects/supervised-learning/regression/logistic-regression/val_features.csv', index=False)
X_test.to_csv('C:/Users/lynst/Documents/GitHub/machine-learning-projects/supervised-learning/regression/logistic-regression/test_features.csv', index=False)

y_train.to_csv('C:/Users/lynst/Documents/GitHub/machine-learning-projects/supervised-learning/regression/logistic-regression/train_labels.csv', index=False)
y_val.to_csv('C:/Users/lynst/Documents/GitHub/machine-learning-projects/supervised-learning/regression/logistic-regression/val_labels.csv', index=False)
y_test.to_csv('C:/Users/lynst/Documents/GitHub/machine-learning-projects/supervised-learning/regression/logistic-regression/test_labels.csv', index=False)

In [ ]:
# Write out cleaned data
updated_df.to_csv('C:/Users/lynst/Documents/GitHub/machine-learning-projects/supervised-learning/regression/logistic-regression/titanic_cleaned.csv', index=False)

#### Scale the Different Ranges of Values
To improve accuracy before modeling by removing unecessary scale or unit measurement differences.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### Logistic Regression Model
#### Train the Data First
This is the stage where I get to fit a model to the data (fit a regression line to the datapoints). Applying a Logistic Regression model to the labeled set and setting the 'multi_class' parameter equal to 'auto' because this will select a binary classifier automatically for the output array. Setting 'multi_class' equal to 'auto' is another way for the model to identify a binary problem to be fit to each label. If it had been a multi-class output then 'auto' would detect that also.

In [ ]:
# import library then I can try different models
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression

logistic_regression = LogisticRegression(multi_class='auto')
log_reg_clf = logistic_regression.fit(X_train, y_train)

Once the classification has been fit predictions can be made. The predict() method predicts the actual class below.

In [ ]:
y_pred = log_reg_clf.predict(X_test)
y_pred

### Evaluation
Now I can evaluate the training set using the score method, which is the coefficient of determination (R-squared) and determines how accurate the regression line is compared to the overall mean.

In [ ]:
log_reg_clf.score(X_train, y_train)

This indicates that having trained a Logistic Regression model on my prepared dataset and having made the predictions above, it's safe to assume it's correct 78.8% of the time. Having calculated all the individual probabilities the score method finds the mean overall probability. 

I can apply this 'score()' method and measure the performance on the validation set.

In [ ]:
log_reg_clf.score(X_val, y_val)

The score for the validation set data can be improved upon. Try removing the "Cabin" feature to see how much information is captured by the model this time. The model obviously needs some fine-tuning before applying it to the unseen test set.

In [ ]:
X = updated_df[['Boarded_Belfast', 'Boarded_Cherbourg', 'Boarded_Queenstown', 'Boarded_Southampton', 'Female', 'Male', 'Pclass',
       'Age_wiki', 'Fare', 'Family_Count']].copy()
X = X[0:889]
X.head()

Train the same model after re-scaling the values using some normalization algorithm first, then re-apply the logistic regression.

#### Re-Scaling

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

logistic_regression = LogisticRegression(multi_class='auto')
log_reg_clf = logistic_regression.fit(X_train, y_train)

log_reg_clf.score(X_train, y_train)

Having dropped the Cabin feature there is a negligable improvement so the model captures a small incremental change in the score having removed the information from this feature. Moving on to preprocessing, it may help to train a Polynomial function to fit the curve more accurately to the data.

### Polynomial Features
Checking the parameters or attributes which can be adjusted using the PolynomialFeatures() model I will first try changing the 'degree' parameter to include a tuple for the 'min_degree' and 'max_degree' values.

In [ ]:
from sklearn import linear_model, preprocessing

logistic_regression = LogisticRegression(multi_class='auto')
log_reg_clf = logistic_regression.fit(X_train, y_train)

polynomial_regression = preprocessing.PolynomialFeatures(degree=(2,4))
poly_features = polynomial_regression.fit_transform(X_train)

log_reg_clf = logistic_regression.fit(poly_features, y_train)

log_reg_clf.score(poly_features, y_train)

Again, the polynomal features model is nudging the score in the right direction by attempting to fit the Sigmoid curve more accurately to the values and reducing the overall error. Next, try an ensemble learning classifier model.

Adding some of the parameters such as penalty and solver provides a slight improvement. L2 regularization or ridge regression which will reduce the individual weights for each observation as close to zero as possible.

In [ ]:
from sklearn import linear_model, preprocessing

logistic_regression = LogisticRegression(penalty='l2', solver='newton-cg', multi_class='auto')
log_reg_clf = logistic_regression.fit(X_train, y_train)

polynomial_regression = preprocessing.PolynomialFeatures(degree=(2,4))
poly_features = polynomial_regression.fit_transform(X_train)

log_reg_clf = logistic_regression.fit(poly_features, y_train)

log_reg_clf.score(poly_features, y_train)

A slight improvement!

### Leave 1-Out
This should only really be used for really small datasets when I could leave one value or observation out for validation after each training iteration but it would take too long for 889 instances.

### Confusion Matrix
Using a Confusion Matrix is a much better approach to measuring accuracy for a Classifier model such as Logistic Regression. This will display 'Actual' v 'Predicted' values. The left hand rows display 'Actual' negative and positive classes, and the colums display 'Predicted' negative and positive classes. It's important to make sure the length of the target series' are exactly the same for actual (y_test) and predicted (y_pred) values. 

It won't work by testing training data (y_train) because these account for 533 total instances, rather than 178 instances in the test set. It's good to play around with the train-test set size hyperparameters in the 'train_test_split' module. Start with test_size=0.2, followed by 0.3 and 0.4.

The same goes for the split between the validation and test set sizes. Try different percentage splits to derive more accurate outcomes in the accuracy score (R-squared) values.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

# training the model using the standard scaler algorithm
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# creating a classifier model by fitting logistic regression to the data points
logistic_regression = LogisticRegression(penalty='l2', solver='newton-cg', multi_class='auto')
log_reg_clf = logistic_regression.fit(X_train,y_train)

# creating y_train_pred variable and predicting the first 5 values:
y_train_pred = log_reg_clf.predict(X_train)
y_train_pred[:5]

### Introducing Evaluation Metrics
Taking a look at a Confusion Martix, Accuracy Score and Receiver Operating Characteristics (Area Under Curve).

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score

conf = confusion_matrix(y_train, y_train_pred)
print(conf)

In [ ]:
from sklearn.metrics import precision_score, recall_score, roc_auc_score

prec = precision_score(y_train, y_train_pred)
print(f"The Precision Accuracy Score is: ", prec)
rec = recall_score(y_train, y_train_pred)
print(f"The Recall Accuracy Score is: ", rec)
roc_auc = roc_auc_score(y_train, y_train_pred)
print(f"The Receiver Operation Characteristic-Area Under Curve Score is: ", roc_auc)

None of these scores are particularly great for prediction purposes! These are all predictions based on the training data, but shouldn't accuracy scores be applied to test data (target class y_test, and predicted class y_pred? What happens if I change the arguments for the confusion_matrix() function to contain the test set?

In [ ]:
from sklearn.metrics import confusion_matrix

conf = confusion_matrix(y_train, y_train_pred)
conf

In [ ]:
# first row:
TN = 293
FP = 40
# second row:
FN = 67
TP = 133

Find the Precision score

In [ ]:
# accuracy of positive predictions
precision = TP / (TP + FP)
precision

Find the recall score

In [ ]:
# ratio of correctly predicted instances
recall = TP / (TP + FN)
recall

Plotting the Actual versus Predicted survival classes using the results from the Confusion Matrix above.

In [ ]:
from sklearn.metrics import plot_confusion_matrix

# plotting actual v predicted values
plt.figure(figsize = (10,7))
plt.title("Actual v Predicted Survival Rates")
plt.xlabel("Predicted")
plt.ylabel("Actual")
sns.heatmap(confusion_matrix(y_train, y_train_pred), annot=True)

Rather than using the confusion_matrix() function, importing separate precision and recall score functions may provide different accuracy results. Let's try!

In [ ]:
from sklearn.metrics import precision_score, recall_score

prec = precision_score(y_train, y_train_pred)
print(f"The Precision Accuracy Score is: ", prec)
rec = recall_score(y_train, y_train_pred)
print(f"The Recall Accuracy Score is: ", rec)

So it's apparent that these methods provide the same scores as the manual calculations in the previous cells. The Accuracy Score measures the proportion of all predictions that were correct.

Precision is the degree to which positive and negative predictions are made accurately, the ratio of true to false positives, or quite simply, the proportion of positive predictions that are correctly classified. Those values which remain represent the values which are incorrectly classified, or the Negative Predicted values. 

The Recall Score relates to the percentage of correctly classified Positive Predictions for each class, or the number of times a particular value (or class) was correctly classified.

Combining both together can be achieved using something called an F1 score for convenience:

In [ ]:
from sklearn.metrics import f1_score

f1 = f1_score(y_train, y_train_pred)
print(f"The F1 Accuracy Score is: ", f1)

This provides a mean value somewhere between the Precision and Recall scores.  Another way to achieve this is by using the predict_proba() method which predicts the probability that each element falls within that particular class.

In [ ]:
log_reg_clf.predict_proba(X_val)

In [ ]:
accuracy_score(y_train, y_train_pred)

This conveys the probability of correctly classifying each individual observation, or simply, the proportion of total predictions that are correct.

There is no direct way to fine-tune or control the 'decision threshold' which is the boundary between the different classifications, but the decision scores can be accessed using the SGDClassifier() model's 'decision_function(). Then it becomes possible to move the decision plane to the left or right in the list of instances to improve the accuracy score's probability.

#### Changing the Test Set Sizes
Repeat the process of train-test-split function by adjusting the test size and other parameters before using different training models each time.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, roc_auc_score, f1_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

# training the model using the standard scaler algorithm
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# creating a classifier model by fitting logistic regression to the data points
logistic_regression = LogisticRegression(penalty='l2', solver='newton-cg', multi_class='auto')
log_reg_clf = logistic_regression.fit(X_train, y_train)

# creating y_train_pred variable and predicting the first 5 values:
y_train_pred = log_reg_clf.predict(X_train)

# summarizing the scores
prec = precision_score(y_train, y_train_pred)
print(f"The Precision Accuracy Score is: ", prec)
rec = recall_score(y_train, y_train_pred)
print(f"The Recall Accuracy Score is: ", rec)
roc_auc = roc_auc_score(y_train, y_train_pred)
print(f"The Receiver Operation Characteristic-Area Under Curve Score is: ", roc_auc)
f1 = f1_score(y_train, y_train_pred)
print(f"The F1 Accuracy Score is: ", f1)

This has improved the accuracy of the model for all scores.

Changing the test_size=0.2 gives:

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, roc_auc_score, f1_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

# training the model using the standard scaler algorithm
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# creating a classifier model by fitting logistic regression to the data points
logistic_regression = LogisticRegression(penalty='l2', solver='newton-cg', multi_class='auto')
log_reg_clf = logistic_regression.fit(X_train, y_train)

# creating y_train_pred variable and predicting the first 5 values:
y_train_pred = log_reg_clf.predict(X_train)

# summarizing the scores
prec = precision_score(y_train, y_train_pred)
print(f"The Precision Accuracy Score is: ", prec)
rec = recall_score(y_train, y_train_pred)
print(f"The Recall Accuracy Score is: ", rec)
roc_auc = roc_auc_score(y_train, y_train_pred)
print(f"The Receiver Operation Characteristic-Area Under Curve Score is: ", roc_auc)
f1 = f1_score(y_train, y_train_pred)
print(f"The F1 Accuracy Score is: ", f1)

The recall_score has improved but not the others. Let's try some more classification models.

### Decision Tree Classifier
There are a few different ways to achieve this.

In [ ]:
# Random Forest Classifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn import tree
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, f1_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

# training the model using the standard scaler algorithm
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# creating a model by fitting stochastic gradient descent learning to the data points
decision_tree_classifier = tree.DecisionTreeClassifier(random_state=42)
dtc = decision_tree_classifier.fit(X_train, y_train)

# creating y_train_pred variable and predicting the first 5 values:
y_train_pred = dtc.predict(X_train)

# summarizing the scores
prec = precision_score(y_train, y_train_pred)
print(f"The Precision Accuracy Score is: ", prec)
rec = recall_score(y_train, y_train_pred)
print(f"The Recall Accuracy Score is: ", rec)
roc_auc = roc_auc_score(y_train, y_train_pred)
print(f"The Receiver Operation Characteristic-Area Under Curve Score is: ", roc_auc)
f1 = f1_score(y_train, y_train_pred)
print(f"The F1 Accuracy Score is: ", f1)

Interesting scores but they all appear to be overfitting so the best way to evaluate this result is to apply a cross validation technique and if all the scores are still high then regularization should be applied. It's probably important to note here that cross validation should probably be applied to our model each time when it comes to evaluating scores and is a useful tool for measuring or evaluating estimator performance.

#### Cross-Validation
Fit and evaluate a basic model using K-Fold Cross Validation. The data will be divided into k subsets and the holdout set will be repeated k number of times until all the data points are used in the model. In this example I am setting k equal to 5 folds, or cv=5. Also I will change the scoring hyperparameter to check for improvements.

In [ ]:
from sklearn.model_selection import cross_val_score

# train the cross validation model using the decision_tree as the first argument, X and y values and iterations
scores = cross_val_score(dtc, X_train, y_train, cv=5, scoring="accuracy", n_jobs=-1)
print("K Fold Scores are: ", scores)

This is extremely useful as cross-validation will output a score based on K number of folds, in this example 5 folds, which is directly controlled by the parameter cv=5 (5 splits is also the default number and is the same as specifying 'cv=None'). When using K-Folds CV the average score is taken so inputing the 5 scores above and calculating the mean gives:

In [ ]:
(0.768 + 0.728 + 0.76612903 + 0.80645161 + 0.73387097) / 5

Which can also be achieved by using the mean method():

In [ ]:
print("The mean score is %0.4f with a standard deviation of %0.4f" % (scores.mean(), scores.std()))

So I can see that 76.05% is slightly worse than the previous score from the 'PolynomialFeatures', but is probably a more accurate reflection of the score() function. The overfitting in the DecisionTree is a result of low bias and high variance so this needs to reach some kind of equilibrium and the only way optimize the solution is to fine-tune hyperparameters in this algorithm such as max_depth, max_features or random_state.

#### Fine-Tuning the Decision Tree and CV
Adjustments to fine-tune include:

train_test_split - 'test_size',

DecisionTreeClassifier - 'criterion', 'splitter', 'max_depth' and 'random_state',

cross_val_score - 'n_jobs', 'scoring' (which includes accuracy, balanced_accuracy, roc_auc, f1 as options for Classification) and 'cv'.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

# training the model using the standard scaler algorithm
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# creating a model by fitting stochastic gradient descent learning to the data points
decision_tree_classifier = tree.DecisionTreeClassifier(criterion="gini", splitter="best", max_depth=None, random_state=None)
dtc = decision_tree_classifier.fit(X_train, y_train)

# creating y_train_pred variable and predicting the first 5 values:
y_train_pred = dtc.predict(X_train)

# summarizing the scores
prec = precision_score(y_train, y_train_pred)
print(f"The Precision Accuracy Score is: ", prec)
rec = recall_score(y_train, y_train_pred)
print(f"The Recall Accuracy Score is: ", rec)
roc_auc = roc_auc_score(y_train, y_train_pred)
print(f"The Receiver Operation Characteristic-Area Under Curve Score is: ", roc_auc)
f1 = f1_score(y_train, y_train_pred)
print(f"The F1 Accuracy Score is: ", f1)

In [ ]:
from sklearn.model_selection import cross_val_score

# train the cross validation model using the decision_tree as the first argument, X and y values and iterations
scores = cross_val_score(dtc, X_train, y_train, cv=5, scoring="accuracy", n_jobs=-1)
print("Mean score of %0.2f with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

Having run the model by manually adjusting the hyper-parameters each time it takes far too long and there are too many combinations of scores to contemplate an optimal solution based on the chosen model. An automated search for the optimal combination of hyper-parameters is definitely the way to go and the best ones to use include GridSearchCV, RandomizedSearchCV, BayesSearchCV, SGDClassifier and differential_evolution.

### Grid Search CV

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, f1_score

# training the model using the standard scaler algorithm
scaler = StandardScaler()
#X_train = scaler.fit_transform(X_train)
#X_test = scaler.transform(X_test)

# initialize the algorithm you want to fine-tune
decision_tree_classifier = tree.DecisionTreeClassifier(criterion="gini", splitter="best", max_depth=None, random_state=None)

grid = {'criterion': ['gini','entropy','log_loss'],
        'splitter': ['best','random'],
        'max_depth': [1,2,3,4],
        'max_features': ['auto','sqrt','log2']}

print('Number of tested models: %i'
     % np.prod([len(grid[element]) for element in grid]))

score_metric = 'accuracy'







dtc = decision_tree_classifier.fit(X_train, y_train)

# create 
'''
dtc_pipe = Pipeline([('scaler', StandardScaler()),
                     ('dtc', DecisionTreeClassifier())])
dtc_pipe.fit(X_train, y_train)
dtc_pipe.score(X_train, y_train)
'''

# create a parameter grid
'''
param_grid = [{'criterion': ['gini','entropy','log_loss'],
                   'splitter': ['best','random'],
                   'max_depth': [0,2,3,4],
                   'random_state': [0,1,42],
                   'max_features': [None,'sqrt','log2']}]
'''

gs_dtc = GridSearchCV(dtc_pipe,
                      param_grid=param_grid,
                      scoring='accuracy',
                      cv=5)
gs_dtc.fit(X_train, y_train)
gs_dtc.best_params_
gs_dtc.best_score_

# initialize the grid search cv function
#grid = GridSearchCV(estimator=dtc, param_grid=param_grid, cv=5)
# fit the grid to our training set
#grid.fit(X_train, y_train)

# to check the best parameters selected by the GridSearchCV function:
print("Optimal parameters %s accuracy score of %0.2f"
      % (gs_dtc.best_params_, gs_dtc.best_score_))

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, f1_score

# training the model using the standard scaler algorithm
scaler = StandardScaler()
#X_train = scaler.fit_transform(X_train)
#X_test = scaler.transform(X_test)

# initialize the algorithm you want to fine-tune
decision_tree_classifier = tree.DecisionTreeClassifier(criterion="gini", splitter="best", max_depth=None, random_state=None)

grid = {'criterion': ['gini','entropy','log_loss'],
        'splitter': ['best','random'],
        'max_depth': [1,2,3,4],
        'max_features': ['auto','sqrt','log2']}

print('Number of tested models: %i'
     % np.prod([len(grid[element]) for element in grid]))

score_metric = 'accuracy'


In [ ]:
from sklearn.model_selection import cross_val_score
print('Baseline with default parameters: %.3f'
     % np.mean(cross_val_score(decision_tree_classifier, X, y
                              cv=5, scoring=score_metric, n_jobs=-1)))

In [ ]:
from sklearn.model_selection import GridSearchCV
search = GridSearchCV(estimator=decision_tree_classifier,
                      param_grid=grid,
                      scoring=score_metric,
                      n_jobs=-1,
                      refit=True,
                      return_train_score=True,
                      cv=5)

search.fit(X,y)

In [ ]:
dtc_pipe.get_params().keys()

In [ ]:
from sklearn.datasets import make_classification

X, Y = make_classification(n_samples=889, n_classes=2, n_features=11, n_redundant=0, random_state=1)

In [ ]:
X.shape, Y.shape

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [ ]:
X_train.shape, Y_train.shape

In [ ]:
X_test.shape, Y_test.shape

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf = RandomForestClassifier(max_features=5, n_estimators=100)

In [ ]:
rf.fit(X_train, Y_train)

In [ ]:
rf.score(X_test, Y_test)

In [ ]:
Y_pred = rf.predict(X_test)

In [ ]:
accuracy_score(Y_pred, Y_test)

In [ ]:
Y_pred, Y_test

In [ ]:
from sklearn.model_selection import GridSearchCV
import numpy as np

max_features_range = np.arange(1,11,1)
n_estimators_range = np.arange(1,889,10)
param_grid = dict(max_features=max_features_range, n_estimators=n_estimators_range)

rf = RandomForestClassifier()

grid = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5)

In [ ]:
grid.fit(X_train, Y_train)

In [ ]:
print("The best parameters are %s with a score of %0.2f"
      % (grid.best_params_, grid.best_score_))

### Random Forest Classifier

In [ ]:
# Random Forest Classifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, roc_auc_score, f1_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

# training the model using the standard scaler algorithm
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# creating a model by fitting stochastic gradient descent learning to the data points
random_forest_classifier = RandomForestClassifier(max_features=10, n_estimators=100)
rfc = random_forest__classifier.fit(X_train, y_train)

# creating y_train_pred variable and predicting the first 5 values:
y_train_pred = rfc.predict(X_train)

# summarizing the scores
prec = precision_score(y_train, y_train_pred)
print(f"The Precision Accuracy Score is: ", prec)
rec = recall_score(y_train, y_train_pred)
print(f"The Recall Accuracy Score is: ", rec)
roc_auc = roc_auc_score(y_train, y_train_pred)
print(f"The Receiver Operation Characteristic-Area Under Curve Score is: ", roc_auc)
f1 = f1_score(y_train, y_train_pred)
print(f"The F1 Accuracy Score is: ", f1)

### SGDClassifier

In [ ]:
# SGD Classifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import precision_score, recall_score, roc_auc_score, f1_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

# training the model using the standard scaler algorithm
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# creating a model by fitting stochastic gradient descent learning to the data points
sgd_classifier = SGDClassifier(alpha=0.001, n_jobs=-1)
sgd = sgd_classifier.fit(X_train, y_train)

# creating y_train_pred variable and predicting the first 5 values:
y_train_pred = sgd.predict(X_train)

# summarizing the scores
prec = precision_score(y_train, y_train_pred)
print(f"The Precision Accuracy Score is: ", prec)
rec = recall_score(y_train, y_train_pred)
print(f"The Recall Accuracy Score is: ", rec)
roc_auc = roc_auc_score(y_train, y_train_pred)
print(f"The Receiver Operation Characteristic-Area Under Curve Score is: ", roc_auc)
f1 = f1_score(y_train, y_train_pred)
print(f"The F1 Accuracy Score is: ", f1)

This seems to produce about as high an f1 score as I can get.

### K-Nearest Neighbors

In [ ]:
# K-Nearest Neighbors
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

# training the model using the standard scaler algorithm
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

k_nearest_neighbors = KNeighborsClassifier(n_neighbors=3)
knc = k_nearest_neighbors.fit(X_train, y_train)

# creating y_train_pred variable and predicting the first 5 values:
y_train_pred = knc.predict(X_train)

# summarizing the scores
prec = precision_score(y_train, y_train_pred)
print(f"The Precision Accuracy Score is: ", prec)
rec = recall_score(y_train, y_train_pred)
print(f"The Recall Accuracy Score is: ", rec)
roc_auc = roc_auc_score(y_train, y_train_pred)
print(f"The Receiver Operation Characteristic-Area Under Curve Score is: ", roc_auc)
f1 = f1_score(y_train, y_train_pred)
print(f"The F1 Accuracy Score is: ", f1)

These scores are consistently higher.

### Support Vector Machine

In [ ]:
# Support Vector Machine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

# training the model using the standard scaler algorithm
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

support_vector_machine = SVC(gamma='scale')
svc = support_vector_machine.fit(X_train, y_train)

# creating y_train_pred variable and predicting the first 5 values:
y_train_pred = svc.predict(X_train)

# summarizing the scores
prec = precision_score(y_train, y_train_pred)
print(f"The Precision Accuracy Score is: ", prec)
rec = recall_score(y_train, y_train_pred)
print(f"The Recall Accuracy Score is: ", rec)
roc_auc = roc_auc_score(y_train, y_train_pred)
print(f"The Receiver Operation Characteristic-Area Under Curve Score is: ", roc_auc)
f1 = f1_score(y_train, y_train_pred)
print(f"The F1 Accuracy Score is: ", f1)

### Naive Bayes Classifiers

In [ ]:
# Gaussian Naive Bayes Classifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

# training the model using the standard scaler algorithm
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

nbc = GaussianNB()
nbc.fit(X_train, y_train)

# creating y_train_pred variable and predicting the first 5 values:
y_train_pred = nbc.predict(X_train)

# summarizing the scores
prec = precision_score(y_train, y_train_pred)
print(f"The Precision Accuracy Score is: ", prec)
rec = recall_score(y_train, y_train_pred)
print(f"The Recall Accuracy Score is: ", rec)
roc_auc = roc_auc_score(y_train, y_train_pred)
print(f"The Receiver Operation Characteristic-Area Under Curve Score is: ", roc_auc)
f1 = f1_score(y_train, y_train_pred)
print(f"The F1 Accuracy Score is: ", f1)

In [ ]:
# XGBClassifier
XGBClassifier

In [ ]:
# GradientBoostingClassifier
GradientBoostingClassifier

In [ ]:
# AdaBoostClassifier
AdaBoostClassifier

In [ ]:
# Convoluted Neural Networks
CNN

### ROC Curve
Looking at the Receiver-Operating-Characteristics will plot the True Positive rate of classification (Sensitivity) against the True Negative, or False-Positive rate (Specificity).

In [ ]:
roc_auc = roc_auc_score(y_test, y_pred)
print(f"The Receiver Operation Characteristic-Area Under Curve Score is: ", roc_auc)

The best score is in the range of 0.9 to 1.0. The score of 0.77 is fair, or average with respect to this model.

In [ ]:
from sklearn.metrics import roc_curve

fpr, tpr, thresholds = roc_curve(y_train, y_scores)

# plot the true-positive rate v false-positive rate
def plot_roc_curve(fpr, tpr, label=None):
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], "k--")
    plt.xlabel("fpr")
    plt.ylabel("tpr")
    
plot_roc_curve(fpr, tpr)
plt.show()

The blue curve should be pushed as far to the top left-hand corner as possible for a more accurate model. This can be improved upon by applying a different model such as a Random Classifier.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict

rf = RandomForestClassifier(random_state=42)
y_prob_rf = cross_val_predict(rf, X_train, y_train, cv=3, method="predict_proba")

y_scores_rf = y_prob_rf[:, :1]
fpr_rf, tpr_rf, thresholds_rf = roc_curve(y_train, y_scores_rf)

plt.plot(fpr, tpr, "b:", label="SGD")
plot_roc_curve(tpr_rf, fpr_rf, "Random Forest Model")
plt.legend(loc="lower right")
plt.show()

So the Random Forest Classifier model has improved the Area Under the Curve because it is better at predicting the target values, or survival rate in this model.

## Evaluation
How does the score from the LogisticRegression() model generalize to the test set?

In [ ]:
log_reg_clf.score(X_test, y_test)

The model is generalizing reasonably well to the test set.

## Improving Evaluation Metrics
Iterating the process with new algorithms and improving the evaluation scores. I will repeat this process by trying out iterations changing Parameters and Hyperparameters.

In [ ]:
# using the features and labels data subsets from earlier
train_features = pd.read_csv('C:/Users/lynst/Documents/GitHub/machine-learning-projects/supervised-learning/regression/logistic-regression/X.csv')
train_labels = pd.read_csv('C:/Users/lynst/Documents/GitHub/machine-learning-projects/supervised-learning/regression/logistic-regression/y.csv', header=None)

Try importing the 'cross_val_score' library and reproducing a variety of validation scores through selection and division of the dataset into 5 equal parts.

In [ ]:
from sklearn.model_selection import cross_val_score

rfc = RandomForestClassifier()

scores = cross_val_score(rfc, train_features, train_labels.values.ravel(), cv=5)
scores

In [ ]:
def print_results(results):
    print('BEST PARAMS: {}\n'.format(results.best_params_))

    means = results.cv_results_['mean_test_score']
    stds = results.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, results.cv_results_['params']):
        print('{} (+/-{}) for {}'.format(round(mean, 3), round(std * 2, 3), params))

In [ ]:
rf = RandomForestClassifier()
parameters = {
    'n_estimators': [5, 50, 100],
    'max_depth': [2, 10, 20, None]
}

cv = GridSearchCV(rf, parameters, cv=5)
cv.fit(tr_features, tr_labels.values.ravel())

print_results(cv)

## Pipeline: Evaluate results on validation set
Using the Titanic dataset from this Kaggle competition.

In this section, we will use what we learned in last section to fit the best few models on the full training set and then evaluate the model on the validation set.

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score

tr_features = pd.read_csv('C:/Users/lynst/Documents/Python Scripts/Ex_Files_Applied_Machine_Learning/Exercise Files/train_features.csv')
tr_labels = pd.read_csv('C:/Users/lynst/Documents/Python Scripts/Ex_Files_Applied_Machine_Learning/Exercise Files/train_labels.csv', header=None)

val_features = pd.read_csv('C:/Users/lynst/Documents/Python Scripts/Ex_Files_Applied_Machine_Learning/Exercise Files/val_features.csv')
val_labels = pd.read_csv('C:/Users/lynst/Documents/Python Scripts/Ex_Files_Applied_Machine_Learning/Exercise Files/val_labels.csv', header=None)

te_features = pd.read_csv('C:/Users/lynst/Documents/Python Scripts/Ex_Files_Applied_Machine_Learning/Exercise Files/test_features.csv')
te_labels = pd.read_csv('C:/Users/lynst/Documents/Python Scripts/Ex_Files_Applied_Machine_Learning/Exercise Files/test_labels.csv', header=None)

In [ ]:
rf1 = RandomForestClassifier(n_estimators=5, max_depth=10)
rf1.fit(tr_features, tr_labels.values.ravel())

rf2 = RandomForestClassifier(n_estimators=100, max_depth=10)
rf2.fit(tr_features, tr_labels.values.ravel())

rf3 = RandomForestClassifier(n_estimators=100, max_depth=None)
rf3.fit(tr_features, tr_labels.values.ravel())

In [ ]:
for mdl in [rf1, rf2, rf3]:
    y_pred = mdl.predict(val_features)
    accuracy = round(accuracy_score(val_labels, y_pred), 3)
    precision = round(precision_score(val_labels, y_pred), 3)
    recall = round(recall_score(val_labels, y_pred), 3)
    print('MAX DEPTH: {} / # OF EST: {} -- A: {} / P: {} / R: {}'.format(mdl.max_depth,
                                                                         mdl.n_estimators,
                                                                         accuracy,
                                                                         precision,
                                                                         recall))

In [ ]:
y_pred = rf2.predict(te_features)
accuracy = round(accuracy_score(te_labels, y_pred), 3)
precision = round(precision_score(te_labels, y_pred), 3)
recall = round(recall_score(te_labels, y_pred), 3)
print('MAX DEPTH: {} / # OF EST: {} -- A: {} / P: {} / R: {}'.format(rf2.max_depth,
                                                                     rf2.n_estimators,
                                                                     accuracy,
                                                                     precision,
                                                                     recall))

In [ ]:
%load_ext watermark

# python, ipython, packages, and machine characteristics
%watermark -v -m -p wget,pandas,numpy,watermark,matplotlib,seaborn,sklearn,warnings

# date
print (" ")
%watermark -u -n -t -z